In [1]:
!pip install pandas
!pip install openpyxl
!pip install requests
!pip install beautifulsoup4
!pip install newspaper3k
!pip install selenium
!pip install openai
!pip install serpapi
!pip install googlesearch-python
!pip install google-search-results
!pip install GoogleNews
!pip install "httpx[http2]"
!pip install "lxml[html_clean]"

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=1e7d902e2c1a11b7308f48dd5c455aa42eafb111cc54f315b6c7cb608cc5a64d
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=89dbb12074a4d097990dade50adf45ddd5cc12747e24b507e3cbc60988b4cc6d
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [2]:
from tabulate import tabulate
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import random
import re
from datetime import datetime
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
import pandas as pd
from urllib.parse import quote_plus
from serpapi.google_search import GoogleSearch
from dateutil import parser


## Data frame unificado


In [3]:
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"


In [13]:
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"

# ===================================================
# 🔹 FUNCIONES DE SCRAPING
# ===================================================

def link_valido(url):
    try:
        r = requests.head(url, timeout=5, allow_redirects=True)
        return r.status_code == 200
    except:
        return False

def buscar_noticias_generales(nombre_fuente, url_base, termino_busqueda, max_resultados=15):
    """Scrapea noticias desde sitios como El Tiempo, ADE o FECODE."""
    print(f"\n Buscando en {nombre_fuente}: {termino_busqueda}")
    datos = []
    termino_codificado = quote_plus(termino_busqueda)
    url_busqueda = f"{url_base}{termino_codificado}"

    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        articulos = soup.find_all('article', limit=max_resultados)
        print(f" Encontrados {len(articulos)} artículos en {nombre_fuente}")

        for art in articulos:
            titulo = (art.find('h2') or art.find('h3'))
            enlace = art.find('a', href=True)

            if not titulo or not enlace:
                continue

            link = enlace['href']
            if not link.startswith('http'):
                link = f"{url_base.rstrip('/')}/{link.lstrip('/')}"

            if not link_valido(link):
                continue

            descripcion = art.find('p').get_text(strip=True) if art.find('p') else ''
            fecha_elem = art.find('time') or art.find('span', class_=re.compile('date')) or art.find('div', class_=re.compile('fecha'))
            if fecha_elem:
                fecha_cruda = fecha_elem.get('datetime') or fecha_elem.get_text(strip=True)
            else:
                fecha_cruda = '2023'
            fecha = fecha_elem.get_text(strip=True) if fecha_elem else 'No especificada'

            datos.append({
                'fuente': nombre_fuente,
                'titulo': titulo.get_text(strip=True),
                'descripcion': descripcion,
                'fecha_publicacion': fecha,
                'url': link,
                'periodo_busqueda': termino_busqueda,
                'fecha_extraccion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    except Exception as e:
        print(f" Error en {nombre_fuente}: {e}")

    time.sleep(random.uniform(1.5, 3.5))
    return datos


def buscar_con_googlenews():
    """Extrae resultados desde GoogleNews API."""
    googlenews = GoogleNews(lang='es', encode='utf-8')
    googlenews.set_time_range('01/01/2023', '12/31/2023')
    resultados = []
    queries = [
            'paro docentes Bogotá',
            'paro maestros Colombia',
            'huelga de profesores Bogotá',
            'protesta Fecode 2023',
            'ADE Bogotá educación',
            # Variaciones con sinónimos y contexto
            'cese de actividades docentes',
            'suspensión de clases Bogotá',
            'manifestación de maestros Colombia',
            'docentes sin clases 2023',
            'reclamos del magisterio colombiano',
            'paro educativo nacional 2023',
            # Variaciones temporales
            'ministerio de educación paro 2023',
            'educadores en huelga 2023',
            'protesta del sindicato de maestros 2023',
            'movilización docente Bogotá 2023'
    ]
    for q in queries:
        googlenews.search(q)
        time.sleep(random.uniform(2, 4))
        res = googlenews.result()

        for r in res:
            url = r.get('link', '')
            fecha_cruda = r.get('date', '2023')
            fecha = limpiar_fecha(fecha_cruda)

            if not url or url in urls_vistas or fecha != "2023":
                continue
            urls_vistas.add(url)
            resultados.append({
             'fuente': r.get('media', 'GoogleNews'),
             'titulo': r.get('title'),
             'descripcion': r.get('desc', ''),
             'url': r.get('link', ''),
             'fecha_publicacion': r.get('date', ''),
             'periodo_busqueda': q,
             'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })
    print(f" GoogleNews: {len(resultados)} resultados")
    return resultados
def limpiar_fecha(texto):
    """Extrae el año de una cadena de texto o fecha ISO; si no hay, devuelve '2023'."""
    import re
    if not texto:
        return "2023"
    años = re.findall(r'20\d{2}', str(texto))
    return años[0] if años else "2023"

def buscar_con_serpapi(api_key):
    """Busca en Google usando SerpAPI y devuelve resultados compatibles."""
    print("\n Buscando con SerpAPI...")
    resultados = []
    queries = [
        '("suspensión de clases" OR "cese de actividades" OR "interrupción académica") Bogotá 2023',
        '("paro de maestros" OR "paro docente" OR "huelga de profesores" OR "protesta de docentes") Bogotá 2023',
        '("interrupción de clases" OR "cancelación de clases" OR "no hay clases") Bogotá 2023',
        '("sindicato de maestros" OR "FECODE" OR "ADE Bogotá") AND (paro OR huelga OR protesta) 2023',
    ]

    for query in queries:
        print(f"  • {query}")
        params = {
            "engine": "google",
            "q": f"{query} after:2023-01-01 before:2023-12-31",
            "api_key": api_key,
            "num": 100,
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            items = results.get("organic_results", [])
            for item in items:
                resultados.append({
                    'fuente': 'Google (SerpAPI)',
                    'titulo': item.get('title'),
                    'fecha_publicacion': item.get('date', ''),
                    'descripcion': item.get('snippet', ''),
                    'url': item.get('link'),
                    'periodo_busqueda': query,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except Exception as e:
            print(f" Error con SerpAPI en query '{query}': {e}")
            time.sleep(random.uniform(1, 2))
    print(f" SerpAPI: {len(resultados)} resultados")
    return resultados
# ===================================================
# 🔹 EJECUCIÓN PRINCIPAL
# ===================================================
if __name__ == "__main__":
    SerpAPI_tkn1 = api_key

    resultados_totales = []
    print("\n Iniciando recolección de noticias del año 2023...")

    fuentes = {
        "El Tiempo": "https://www.eltiempo.com/buscar/",
        "El Espectador": "https://www.elespectador.com/buscador/",
        "Semana": "https://www.semana.com/buscador/?query=",
        "RCN Radio": "https://www.rcnradio.com/buscar?query=",
        "Noticias Uno": "https://canal1.com.co/buscar/?q=",
        "FECODE": "https://fecode.edu.co/?s=",
        "ADE Bogotá": "https://adebogota.org/index.php/buscar?q=",
        "Portafolio": "https://www.portafolio.co/buscar?q=",
        "SED": "https://www.educacionbogota.edu.co/noticias?field_noticia_descripcion_value=",
        "Bogota.gov": "https://bogota.gov.co/tag/ingreso-minimo-garantizado?search_api_fulltext=",
        "El colombiano": "https://www.elcolombiano.com/busquedas/-/search/"
    }

    terminos_busqueda = [
        "paro docentes",
        "paro maestros",
        "huelga de profesores",
        "protesta docente 2023",
        "cese de actividades escolares 2023",
        "manifestación de educadores 2023",
        "Fecode paro",
        "ADE Bogotá 2023",
        "sindicato de maestros 2023",
        "magisterio colombiano 2023",
        "educadores ADE Fecode 2023",
        "reforma educativa 2023 Colombia",
        "clases suspendidas Bogotá",
        "negociaciones Fecode Gobierno",
        "condiciones laborales docentes 2023"
    ]

    for fuente, url_base in fuentes.items():
        for termino in terminos_busqueda:
            noticias = buscar_noticias_generales(fuente, url_base, termino)

            noticias_filtradas = []
            for n in noticias:
                fecha = n.get("fecha_publicacion", "")
                años = re.findall(r'20\d{2}', str(fecha))
                if "2023" in años or fecha == "2023":
                    noticias_filtradas.append(n)
        resultados_totales.extend(noticias_filtradas)
        time.sleep(random.uniform(2, 4))

    # --- GoogleNews ---
    resultados_totales.extend(buscar_con_googlenews())

    # --- SerpAPI ---
    resultados_totales.extend(buscar_con_serpapi(SerpAPI_tkn1))

    # ===================================================
    # 🔹 Unir todo y guardar CSV
    # ===================================================
    df = pd.DataFrame(resultados_totales)
    df = df[df["url"].str.contains("http", na=False)]
    df = df.drop_duplicates(subset='url', keep='first')
df.to_csv("noticias_paros_docentes_2023.csv", index=False, encoding="utf-8-sig")

print(f"\n Total de noticias guardadas: {len(df)}")
print(" Archivo: noticias_paros_docentes_2023.csv")
print("\n PROCESO COMPLETADO CON ÉXITO ")



 Iniciando recolección de noticias del año 2023...

 Buscando en El Tiempo: paro docentes 
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: paro maestros 
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: huelga de profesores 
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: protesta docente 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: cese de actividades escolares 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: manifestación de educadores 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: Fecode paro 
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: ADE Bogotá 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: sindicato de maestros 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: magisterio colombiano 2023
 Encontrados 15 artículos en El Tiempo

 Buscando en El Tiempo: educadores ADE Fecode 2023
 Encontrados 15 artículos en El

NameError: name 'urls_vistas' is not defined

In [15]:

# ===================================================
# 🔹 API KEY
# ===================================================
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"

# ===================================================
# 🔹 FUNCIONES AUXILIARES
# ===================================================

def link_valido(url):
    """Verifica si un enlace responde correctamente."""
    try:
        r = requests.head(url, timeout=5, allow_redirects=True)
        return r.status_code == 200
    except:
        return False


def normalizar_fecha(texto):
    """Convierte una fecha en formato normalizado YYYY-MM-DD."""
    if not texto:
        return "2023-01-01"
    try:
        fecha = parser.parse(texto, fuzzy=True)
        return fecha.strftime("%Y-%m-%d")
    except:
        return "2023-01-01"


def obtener_fecha_segura(elemento_html):
    """Intenta obtener la fecha desde el atributo datetime o el texto del elemento."""
    if not elemento_html:
        return "2023-01-01"
    if elemento_html.has_attr('datetime'):
        return normalizar_fecha(elemento_html['datetime'])
    return normalizar_fecha(elemento_html.get_text(strip=True))


# ===================================================
# 🔹 FUNCIONES DE SCRAPING
# ===================================================

def buscar_noticias_generales(nombre_fuente, url_base, termino_busqueda, max_resultados=15):
    """Scrapea noticias desde medios específicos."""
    print(f"\n🔍 Buscando en {nombre_fuente}: {termino_busqueda}")
    datos = []
    urls_vistas = set()
    termino_codificado = quote_plus(termino_busqueda)
    url_busqueda = f"{url_base}{termino_codificado}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        articulos = soup.find_all('article', limit=max_resultados)
        print(f"📰 Encontrados {len(articulos)} artículos en {nombre_fuente}")

        for art in articulos:
            titulo = (art.find('h2') or art.find('h3'))
            enlace = art.find('a', href=True)
            if not titulo or not enlace:
                continue

            link = enlace['href']
            if not link.startswith('http'):
                link = f"{url_base.rstrip('/')}/{link.lstrip('/')}"

            if link in urls_vistas or not link_valido(link):
                continue
            urls_vistas.add(link)

            descripcion = art.find('p').get_text(strip=True) if art.find('p') else ''
            fecha_elem = art.find('time') or art.find('span', class_=re.compile('date')) or art.find('div', class_=re.compile('fecha'))
            fecha_publicacion = obtener_fecha_segura(fecha_elem)

            # 🔹 Filtrar solo noticias del año 2023
            if not fecha_publicacion.startswith("2023"):
                continue

            datos.append({
                'fuente': nombre_fuente,
                'titulo': titulo.get_text(strip=True),
                'descripcion': descripcion,
                'fecha_publicacion': fecha_publicacion,
                'url': link,
                'periodo_busqueda': termino_busqueda,
                'fecha_extraccion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    except Exception as e:
        print(f"❌ Error en {nombre_fuente}: {e}")

    time.sleep(random.uniform(1.5, 3.5))
    return datos


def buscar_con_googlenews():
    """Extrae resultados desde GoogleNews API (solo 2023)."""
    googlenews = GoogleNews(lang='es', encode='utf-8')
    googlenews.set_time_range('01/01/2023', '12/31/2023')
    resultados = []
    urls_vistas = set()

    queries = [
        'paro docentes Bogotá',
        'paro maestros Colombia',
        'huelga de profesores Bogotá',
        'protesta Fecode 2023',
        'ADE Bogotá educación',
        'cese de actividades docentes',
        'suspensión de clases Bogotá',
        'manifestación de maestros Colombia',
        'reclamos del magisterio colombiano',
        'paro educativo nacional 2023',
        'ministerio de educación paro 2023'
    ]

    for q in queries:
        googlenews.search(q)
        time.sleep(random.uniform(2, 4))
        res = googlenews.result()

        for r in res:
            url = r.get('link', '')
            if not url or url in urls_vistas:
                continue
            urls_vistas.add(url)

            fecha_cruda = r.get('date', '2023-01-01')
            fecha_publicacion = normalizar_fecha(fecha_cruda)
            if not fecha_publicacion.startswith("2023"):
                continue

            resultados.append({
                'fuente': r.get('media', 'GoogleNews'),
                'titulo': r.get('title'),
                'descripcion': r.get('desc', ''),
                'url': url,
                'fecha_publicacion': fecha_publicacion,
                'periodo_busqueda': q,
                'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

    print(f"✅ GoogleNews: {len(resultados)} resultados únicos (solo 2023)")
    return resultados


def buscar_con_serpapi(api_key):
    """Busca en Google usando SerpAPI y devuelve resultados compatibles (solo 2023)."""
    print("\n🔍 Buscando con SerpAPI...")
    resultados = []
    urls_vistas = set()
    queries = [
        '("suspensión de clases" OR "cese de actividades" OR "interrupción académica") Bogotá 2023',
        '("paro de maestros" OR "paro docente" OR "huelga de profesores" OR "protesta de docentes") Bogotá 2023',
        '("interrupción de clases" OR "cancelación de clases" OR "no hay clases") Bogotá 2023',
        '("sindicato de maestros" OR "FECODE" OR "ADE Bogotá") AND (paro OR huelga OR protesta) 2023',
    ]

    for query in queries:
        print(f"  • {query}")
        params = {
            "engine": "google",
            "q": f"{query} after:2023-01-01 before:2023-12-31",
            "api_key": api_key,
            "num": 100,
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            items = results.get("organic_results", [])
            for item in items:
                url = item.get('link', '')
                if not url or url in urls_vistas:
                    continue
                urls_vistas.add(url)

                resultados.append({
                    'fuente': 'Google (SerpAPI)',
                    'titulo': item.get('title'),
                    'descripcion': item.get('snippet', ''),
                    'url': url,
                    'fecha_publicacion': "2023-01-01",
                    'periodo_busqueda': query,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except Exception as e:
            print(f"❌ Error con SerpAPI en query '{query}': {e}")
            time.sleep(random.uniform(1, 2))
    print(f"✅ SerpAPI: {len(resultados)} resultados únicos (solo 2023)")
    return resultados


# ===================================================
# 🔹 EJECUCIÓN PRINCIPAL
# ===================================================
if __name__ == "__main__":
    SerpAPI_tkn1 = api_key
    resultados_totales = []
    print("\n🚀 Iniciando recolección de noticias del año 2023...")

    fuentes = {
        "El Tiempo": "https://www.eltiempo.com/buscar/",
        "El Espectador": "https://www.elespectador.com/buscador/",
        "FECODE": "https://fecode.edu.co/?s=",
        "ADE Bogotá": "https://adebogota.org/index.php/buscar?q=",
        "Portafolio": "https://www.portafolio.co/buscar?q=",
        "SED": "https://www.educacionbogota.edu.co/noticias?field_noticia_descripcion_value=",
        "Bogota.gov": "https://bogota.gov.co/tag/ingreso-minimo-garantizado?search_api_fulltext=",
        "El colombiano": "https://www.elcolombiano.com/busquedas/-/search/",
        "Caracol" : "https://www.noticiascaracol.com/busqueda?q=",
        "Valora Analitik" : "https://www.valoraanalitik.com/?s="
    }

    terminos_busqueda = [
        "paro docentes",
        "paro maestros",
        "huelga de profesores",
        "protesta docente 2023",
        "cese de actividades escolares 2023",
        "manifestación de educadores 2023",
        "Fecode paro",
        "sindicato de maestros 2023",
        "reforma educativa 2023 Colombia",
        "clases suspendidas Bogotá"
    ]

    for fuente, url_base in fuentes.items():
        for termino in terminos_busqueda:
            noticias = buscar_noticias_generales(fuente, url_base, termino)
            resultados_totales.extend(noticias)
            time.sleep(random.uniform(2, 4))

    # --- GoogleNews ---
    resultados_totales.extend(buscar_con_googlenews())

    # --- SerpAPI ---
    resultados_totales.extend(buscar_con_serpapi(SerpAPI_tkn1))

    # ===================================================
    # 🔹 Guardar CSV limpio
    # ===================================================
    df = pd.DataFrame(resultados_totales)
    df = df[df["url"].str.contains("http", na=False)]
    df = df.drop_duplicates(subset='url', keep='first')
    df = df[df["fecha_publicacion"].str.startswith("2023", na=False)]

    df.to_csv("noticias_paros_docentes_2023.csv", index=False, encoding="utf-8-sig")

    print(f"\n✅ Total de noticias guardadas: {len(df)}")
    print("📁 Archivo: noticias_paros_docentes_2023.csv")
    print("\n🟢 PROCESO COMPLETADO CON ÉXITO")



🚀 Iniciando recolección de noticias del año 2023...

🔍 Buscando en El Tiempo: paro docentes
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: paro maestros
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: huelga de profesores
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: protesta docente 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: cese de actividades escolares 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: manifestación de educadores 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: Fecode paro
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: ADE Bogotá 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: sindicato de maestros 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: magisterio colombiano 2023
📰 Encontrados 15 artículos en El Tiempo

🔍 Buscando en El Tiempo: educadores ADE Fecode 2023
📰 Encontrados

In [17]:
# ===================================================
# 🔹 IMPORTACIONES NECESARIAS
# ===================================================
import requests
import pandas as pd
import time
import random
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from GoogleNews import GoogleNews
from serpapi import GoogleSearch
from dateutil import parser
import re

# ===================================================
# 🔹 API KEY
# ===================================================
api_key = "3e815855bc007210de64f13b41d3a9dd56da0540c5da030c9b4baf6142e0c830"

# ===================================================
# 🔹 FUNCIONES AUXILIARES
# ===================================================

def normalizar_fecha(texto):
    """Convierte una fecha en formato normalizado YYYY-MM-DD."""
    if not texto:
        return "2023-01-01"
    try:
        fecha = parser.parse(texto, fuzzy=True)
        return fecha.strftime("%Y-%m-%d")
    except:
        return "2023-01-01"


def link_valido(url):
    try:
        r = requests.head(url, timeout=5, allow_redirects=True)
        return r.status_code == 200
    except:
        return False


def obtener_fecha_segura(elemento_html):
    if not elemento_html:
        return "2023-01-01"
    if elemento_html.has_attr('datetime'):
        return normalizar_fecha(elemento_html['datetime'])
    return normalizar_fecha(elemento_html.get_text(strip=True))


# ===================================================
# 🔹 SCRAPER DE FUENTES OFICIALES
# ===================================================

def buscar_noticias_oficiales(nombre_fuente, url_base, termino_busqueda, max_resultados=25):
    """Scrapea noticias directamente desde sitios oficiales."""
    print(f"\n🏛️ Buscando en {nombre_fuente}: {termino_busqueda}")
    datos = []
    urls_vistas = set()
    termino_codificado = quote_plus(termino_busqueda)
    url_busqueda = f"{url_base}{termino_codificado}"

    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        response = requests.get(url_busqueda, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        articulos = soup.find_all(['article', 'div', 'li'], limit=max_resultados)
        print(f"🔎 {len(articulos)} artículos encontrados en {nombre_fuente}")

        for art in articulos:
            titulo = (art.find('h2') or art.find('h3'))
            enlace = art.find('a', href=True)
            if not titulo or not enlace:
                continue

            link = enlace['href']
            if not link.startswith('http'):
                link = f"{url_base.rstrip('/')}/{link.lstrip('/')}"

            if link in urls_vistas:
                continue
            urls_vistas.add(link)

            descripcion = art.find('p').get_text(strip=True) if art.find('p') else ''
            fecha_elem = art.find('time') or art.find('span', class_=re.compile('date')) or art.find('div', class_=re.compile('fecha'))
            fecha_publicacion = obtener_fecha_segura(fecha_elem)

            if not fecha_publicacion.startswith("2023"):
                continue

            datos.append({
                'fuente': nombre_fuente,
                'titulo': titulo.get_text(strip=True),
                'descripcion': descripcion,
                'fecha_publicacion': fecha_publicacion,
                'url': link,
                'periodo_busqueda': termino_busqueda,
                'fecha_extraccion': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    except Exception as e:
        print(f"❌ Error en {nombre_fuente}: {e}")

    time.sleep(random.uniform(1.5, 3.5))
    return datos


# ===================================================
# 🔹 GOOGLE NEWS + SERPAPI (FILTRADOS A DOMINIOS OFICIALES)
# ===================================================

def buscar_con_serpapi_oficial(api_key):
    print("\n🔍 Buscando con SerpAPI (solo dominios oficiales)...")
    resultados = []
    urls_vistas = set()

    queries = [
        'site:fecode.edu.co "paro docente" OR "huelga" OR "educadores" 2023',
        'site:adebogota.org "paro docente" OR "manifestación" 2023',
        'site:educacionbogota.edu.co "suspensión de clases" OR "protesta" 2023',
        'site:mineducacion.gov.co "docentes" OR "magisterio" 2023',
        'site:bogota.gov.co "educación" OR "paro" 2023'
    ]

    for query in queries:
        print(f"  • {query}")
        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "num": 100,
        }
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            items = results.get("organic_results", [])
            for item in items:
                url = item.get('link', '')
                if not url or url in urls_vistas:
                    continue
                urls_vistas.add(url)

                resultados.append({
                    'fuente': 'Google (SerpAPI)',
                    'titulo': item.get('title'),
                    'descripcion': item.get('snippet', ''),
                    'url': url,
                    'fecha_publicacion': "2023-01-01",
                    'periodo_busqueda': query,
                    'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        except Exception as e:
            print(f"❌ Error con SerpAPI en query '{query}': {e}")
        time.sleep(random.uniform(2, 4))

    print(f"✅ SerpAPI: {len(resultados)} resultados oficiales.")
    return resultados


# ===================================================
# 🔹 EJECUCIÓN PRINCIPAL (MODO OFICIAL AMPLIADO)
# ===================================================

if __name__ == "__main__":
    resultados_totales = []
    print("\n🚀 Recolección extendida de fuentes oficiales (meta: 300+)...")

    fuentes_oficiales = {
        "El Tiempo": "https://www.eltiempo.com/buscar/",
        "El Espectador": "https://www.elespectador.com/buscador/",
        "FECODE": "https://fecode.edu.co/?s=",
        "ADE Bogotá": "https://adebogota.org/index.php/buscar?q=",
        "Portafolio": "https://www.portafolio.co/buscar?q=",
        "SED": "https://www.educacionbogota.edu.co/noticias?field_noticia_descripcion_value=",
        "Bogota.gov": "https://bogota.gov.co/tag/ingreso-minimo-garantizado?search_api_fulltext=",
        "El colombiano": "https://www.elcolombiano.com/busquedas/-/search/",
        "Caracol" : "https://www.noticiascaracol.com/busqueda?q=",
        "Valora Analitik" : "https://www.valoraanalitik.com/?s="
    }

    terminos_busqueda = [
        "paro docentes",
        "paro maestros",
        "huelga de profesores",
        "protesta docente 2023",
        "cese de actividades escolares 2023",
        "manifestación de educadores 2023",
        "Fecode paro",
        "sindicato de maestros 2023",
        "reforma educativa 2023 Colombia",
        "clases suspendidas Bogotá"
    ]

    # --- Scraping directo ---
    for fuente, url_base in fuentes_oficiales.items():
        for termino in terminos_busqueda:
            noticias = buscar_noticias_oficiales(fuente, url_base, termino, max_resultados=40)
            resultados_totales.extend(noticias)
            time.sleep(random.uniform(1.5, 3.0))

    # --- SerpAPI especializado ---
    resultados_totales.extend(buscar_con_serpapi_oficial(api_key))

    # --- Limpieza final ---
    df_oficiales = pd.DataFrame(resultados_totales)
    df_oficiales = df_oficiales[df_oficiales["url"].str.contains("http", na=False)]
    df_oficiales = df_oficiales.drop_duplicates(subset='url', keep='first')

    # Filtrar solo 2023
    df_oficiales = df_oficiales[df_oficiales["fecha_publicacion"].str.startswith("2023", na=False)]

    # --- Guardar ---
    df_oficiales.to_csv("noticias_paros_docentes_2023_oficiales_ampliado.csv", index=False, encoding="utf-8-sig")

    print(f"\n✅ Total de noticias oficiales guardadas: {len(df_oficiales)}")
    print("📁 Archivo: noticias_paros_docentes_2023_oficiales_ampliado.csv")
    print("\n🟢 PROCESO COMPLETADO CON ÉXITO")

    display(df_oficiales.sample(min(10, len(df_oficiales))))



🚀 Recolección extendida de fuentes oficiales (meta: 300+)...

🏛️ Buscando en El Tiempo: paro docentes
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: paro maestros
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: huelga de profesores
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: protesta docente 2023
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: cese de actividades escolares 2023
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: manifestación de educadores 2023
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: Fecode paro
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: sindicato de maestros 2023
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: reforma educativa 2023 Colombia
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Tiempo: clases suspendidas Bogotá
🔎 40 artículos encontrados en El Tiempo

🏛️ Buscando en El Espectador: p

,fuente,titulo,descripcion,fecha_publicacion,url,periodo_busqueda,fecha_extraccion
64,Google (SerpAPI),ACTA DE ACUERDOS 2023,El gobierno ha convocado los diálogos Poder Pe...,2023-01-01,https://fecode.edu.co/images/comunicados/2023/...,"site:fecode.edu.co ""paro docente"" OR ""huelga"" ...",2025-10-23 16:05:59
99,Google (SerpAPI),Foro Educativo Distrital 2023: Ideas que trans...,La Secretaría de Educación del Distrito (SED) ...,2023-01-01,https://bogota.gov.co/mi-ciudad/educacion/foro...,"site:bogota.gov.co ""educación"" OR ""paro"" 2023",2025-10-23 16:06:19
47,Caracol,,,2023-01-01,https://www.noticiascaracol.com/colombia,paro docentes,2025-10-23 16:02:25
107,Google (SerpAPI),Fechas de la semana de receso escolar 2023 en ...,¡Te contamos las fechas! Fechas de la semana d...,2023-01-01,https://bogota.gov.co/mi-ciudad/educacion/fech...,"site:bogota.gov.co ""educación"" OR ""paro"" 2023",2025-10-23 16:06:19
86,Google (SerpAPI),Segundo Comite Distrital de Convivencia Escolar,(asociadas a la protesta) 4. Enfrentamiento ma...,2023-01-01,https://oce.educacionbogota.edu.co/sites/defau...,"site:educacionbogota.edu.co ""suspensión de cla...",2025-10-23 16:06:11
96,Google (SerpAPI),La llama del compañerismo inauguró los Juegos ...,La llama del compañerismo inauguró los Juegos ...,2023-01-01,https://www.mineducacion.gov.co/portal/salapre...,"site:mineducacion.gov.co ""docentes"" OR ""magist...",2025-10-23 16:06:15
94,Google (SerpAPI),"Decretos incrementos salariales para docentes,...",Conozca los decretos que establecen los increm...,2023-01-01,https://www.mineducacion.gov.co/portal/Ministe...,"site:mineducacion.gov.co ""docentes"" OR ""magist...",2025-10-23 16:06:15
69,Google (SerpAPI),CIRCULAR No. de 2023 PARA,Es importante aclarar que los educadores que s...,2023-01-01,https://www.fecode.edu.co/images/comunicados/2...,"site:fecode.edu.co ""paro docente"" OR ""huelga"" ...",2025-10-23 16:05:59
1,FECODE,Circulares 2023,"byfecodeAdminWP8|Abr 7, 2025",2023-01-01,https://fecode.edu.co/circulares-2023/,paro docentes,2025-10-23 15:57:08
73,Google (SerpAPI),carlos fernando galán y la prohibición del der...,"Seguiremos con la manifestación pacífica, denu...",2023-01-01,https://www.adebogota.org/images/pdf/2025/comu...,"site:adebogota.org ""paro docente"" OR ""manifest...",2025-10-23 16:06:04
